# 1. Get data from Azure SQL DB

In [ ]:
import pyodbc

server = 'dataproject.database.windows.net'
database = 'Data515Project'
username = 'uberandtaxinyc'
password = 'Lzcnhwzcylyw515!'
driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

apr_data = []
prepare_statement = 'SELECT [neighborhood], count([Date Time]) as count\
                    FROM [dbo].[uber_04_2014_sample] \
                    GROUP BY [neighborhood]'

cursor.execute("SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [day] = 14")
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

In [ ]:
SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [day] = ??
SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [day] = ?? and [hour] = ??
SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [hour] = ?? 
SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [minute] = ?? 
SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [day] = ?? and [hour] = ?? and [minute] = ?? group by [neighborhood]

#SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [month] = ??


def count_by_query(month,hour):
	'''
	month: the corresponding table name. i.e. apr
	day: int
	hour: int
	minute: int
	'''
	prepare_statement = "SELECT count(*) as count, [neighborhood] FROM [dbo].[" + month + "_time] WHERE [hour] = " 
	+ str(hour) + "group by [neighborhood]"
	cursor.execute(prepare_statement)
	rows = cursor.fetchone()
	#lat_list = []
	#lon_list = []
	neighborhood_list = []
	count_list = []
	for row in rows:
		#lat_list.append(row.LAT)
		#lon_list.append(row.LON)
		neighborhood.append(row.neighborhood)
		count_list.append(row.count)

	dics = {}
	keys = [i for i in neighborhood_list]


	for key in keys:
		dics[i] = count_list[i]

	return dics

# 2. Import cleaned NYC shape data

In [1]:
import json

with open('../data/NYC_Shapes_Cleaned.json') as f:
    data = json.load(f)
nyc_neighborhoods = data

In [2]:
len(nyc_neighborhoods)

195

# 3. Build Bokeh heat map

In [3]:
import numpy as np
import pandas as pd

import bokeh.layouts
from bokeh.io import curdoc, output_file, show
from bokeh.layouts import widgetbox
from bokeh.models import (
    BoxSelectTool,
    Circle, 
    ColumnDataSource,
    CustomJS,
    DataRange1d, 
    GMapPlot,
    GMapOptions,
    HoverTool,
    LogColorMapper,
    PanTool,
    Slider,
    WheelZoomTool,
)
from bokeh.models.glyphs import Patches
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

In [15]:
palette.reverse()

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]

neighborhood_names = list(nyc_neighborhoods.keys())
neighborhood_counts = [np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods)),
                      np.random.randint(100, size = len(nyc_neighborhoods))
                      ]
hour = np.zeros(len(nyc_neighborhoods), dtype=int).tolist()

color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x = neighborhood_xs,
    y = neighborhood_ys,
    name = neighborhood_names,
    hr = hour,
    values = neighborhood_counts,
    value = neighborhood_counts[min(hour)]
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

plot = figure(title="Uber Pickup Distribution", tools=TOOLS, 
              x_axis_location=None, y_axis_location=None)

plot.grid.grid_line_color = None

plot.patches('x', 'y', source=source,
              fill_color={'field': 'value', 'transform': color_mapper},
              fill_alpha=0.7, line_color="black", line_width=0.5)

hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Number of pickups", "@value"),
    ("Hour", "@hr"),
    ("(Long, Lat)", "($x, $y)"),
]

C:\Users\HAOWEN-PC\Anaconda3\lib\site-packages\bokeh\models\sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


In [16]:
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    
    hour = data['hr']
    
    for (i = 0; i < hour.length; i++) {
        hour[i] = f
    }
    values = data['values']
    data['value'] = values[f]
    source.trigger('change');
""")

hour_slider = Slider(start=0, end=23, step=1, title="Hour", callback = callback)
hour_slider.js_on_change('start', callback)

checkbox_button_group = CheckboxButtonGroup(labels=["Uber", "Taxi"], active=[0, 1])

select1 = Select(title="Month:", value="Month", options=["April", "May", "June", "July", "August", "September"])
select2 = Select(title="Date:", value="Date", options=["1", "2", "3", "4", "5", "6", "7"])

In [17]:
curdoc().clear()
show(bokeh.layouts.row(plot, checkbox_button_group, widgetbox(hour_slider, select1, select2, width=300)))

In [10]:
hour

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]